In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")
sys.path.insert(0, "../../../src/")
sys.path.insert(0, "../../../projects/skimpy_bench/src/")
sys.path.insert(0, "../../../projects/skimpy_blox/")

In [2]:
import random
import numpy as np
import skimpy
import skimpy_3d
from skimpy_blox import minecraft, colors
from skimpy import functional as F
import ipywebrtc


In [3]:
%%time
# level = minecraft.SkimpyMinecraftLevel.load("../data/sample_minecraft_world.pickle.gz")
level = minecraft.SkimpyMinecraftLevel.load("../data/Hogwarts1.0.pickle.gz")

CPU times: user 579 ms, sys: 48.3 ms, total: 627 ms
Wall time: 627 ms


In [4]:
%%time
t = level.megatensor().min(1).to(float)
# t = level.chunk_list[0].tensor

CPU times: user 26.9 s, sys: 1.63 s, total: 28.6 s
Wall time: 3.25 s


In [5]:
def blur(tensor):
    G_x = (1 / 64.0) * skimpy.Tensor.from_list([[[1, 6, 15, 20, 15, 6, 1]]]).to(float)
    G_y = (1 / 64.0) * skimpy.Tensor.from_list([[[1], [6], [15], [20], [15], [6], [1]]]).to(float)
    G_z = (1 / 64.0) * skimpy.Tensor.from_list([[[1]], [[6]], [[15]], [[20]], [[15]], [[6]], [[1]]]).to(float)
    tensor = F.conv_3d(tensor, G_x, padding=(3, 0, 0), fill=0.0)
    tensor = F.conv_3d(tensor, G_y, padding=(0, 3, 0), fill=0.0)  
    tensor = F.conv_3d(tensor, G_z, padding=(0, 0, 3), fill=0.0)
    return tensor

In [6]:
for i in range(5):
    t = blur(t)

In [12]:
config = skimpy_3d.VoxelConfig()
config[0] = skimpy_3d.EmptyVoxel()
config[1] = skimpy_3d.ColorVoxel(255, 255, 255)

In [12]:
a  = skimpy.convolve.pad_3d_tensor(t, 2, 0)

In [13]:
%%time
# mesh = skimpy_3d.generate_mesh(config, (t > 0.5).to(int)._tensor)
mesh = skimpy_3d.marching_cubes(a._tensor, 0.5)

IndexError: _Map_base::at

In [10]:
a.shape

(2450, 1090, 258)

In [11]:
t.shape

(2448, 1088, 256)

In [ ]:
from pythreejs import *
import numpy as np
from IPython.display import display

In [ ]:
%%time 

geometry = BufferGeometry(
    attributes={
        "position": BufferAttribute(
            np.array(mesh.positions, dtype='float32').reshape(-1, 3),
            normalized=False,
        ),
        "normal": BufferAttribute(
            np.array(mesh.normals, dtype='float32').reshape(-1, 3),
            normalized=False,
        ),
        "color": BufferAttribute(
            np.array(mesh.colors, dtype='uint8').reshape(-1, 4)
        ),
        "index": BufferAttribute(
            np.array(mesh.triangles, dtype='uint32'),
            normalized=False,
        ),
    },
)

In [ ]:
js_mesh = Mesh(
    geometry=geometry,
    material=MeshLambertMaterial(vertexColors="VertexColors"),
    position=[0, 0, 0],
)

In [ ]:
js_mesh

In [ ]:
camera_pos = level.xyz_to_skimpy_col(level.dense_dimensions())
camera = PerspectiveCamera(position=camera_pos, lookAt=(0,0,0), fov=20)
up = [0, 0, 0]
up[level.column_order[1]] = 1
camera.up = tuple(up)

In [ ]:
light_position = level.xyz_to_skimpy_col(tuple(e / 2 for e in level.dense_dimensions()))
point_light = PointLight(color='#ffffff', position=light_position)
global_light = AmbientLight(color='#333333')

#normals_helper = VertexNormalsHelper(js_mesh, size=0.5, color="#ffcc00")

scene = Scene(children=[js_mesh, camera, point_light, global_light])

In [ ]:
renderer = Renderer(
    camera=camera, 
    background="#b0b0b0", 
    background_opacity=1,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=984,
    height=984,
)

display(renderer)